## TODO:
### Add components
- H
- O2
- NAG
- HOOH
- Ions
- Water

### Fix FAD-RES bonding commands
- Cannot bond ATOMs between different UNITs
    - Solution is to combine into single UNIT then add bond (recommended)
    - Could also define a linker ATOM instead (not recommended)
        - See Ambertools docs under RESIDUE
        - See tleaap help for 'set' command
            - set RESIDUE connect0 ATOM
            - set UNIT.1 connect2 UNIT.1.1

### Review/refactor tleap script
- Verify atom types
- Uncomment solvent box

### Reparameterize FAD

### Verify protonations

### Verify hetero bond-orders

### Handle terminal residues

### Replace RCSB model with PDB-REDO

# Raw Structuraal Files
## CHARMM-GUI
The output of each CHARMM-GUI job was compressed and labeled by JobID in <./data/charmm-gui>.        
See the [README](./data/charmm-gui/README.md) for a list of arguments to replicate each job.         
        
Job state is saved by CHARMM-GUI [¿for two months?].       
To reload a job, provide a JobID to charm-gui.org in the query string.        

http://www.charmm-gui.org/?doc=input/pdbreader&jobid=8606269647&project=pdbreader

### PDB Reader
I used PDB Reader as a quick way to:
- Extract heterogens into separate files
- Clean up atoms/bonds/residues and names
- Model missing sequences

The PDB Reader outputs I use are:
- None of the CHARMM coords or params
- None of the crystallographic waters
- Only molecules extracted from the input 3VTE
    - Apo-protein
    - FAD cofactor
    - NAG glycans

# Initial Structural Files
Apo-protein and heterogen structures used to assemble the initial complex were extracted from [PDBID: 3VTE] using CHARMM-GUI PDB Reader and copied to the [/structures] directory.

> [PDB Reader JobID: 8606269647](http://www.charmm-gui.org/?doc=input/pdbreader&jobid=8606269647&project=pdbreader)
>> 
>> 3VTE apo-protein with missing residues replaced
>> - 3vte_proa.model.pdb [View Structure](http://www.charmm-gui.org/?doc=visualization.ngl&jobid=8606269647&filename=3vte_proa.pdb&project=pdbreader&arg=1)
>>
>> FAD cofactor extracted from 3VTE
>> - 3vte_heta.pdb [View Structue](http://www.charmm-gui.org/?doc=visualization.ngl&jobid=8606269647&filename=3vte_heta.pdb&project=pdbreader&arg=1)
>>
>> NAG glycans extracted from 3VTE
>> - 3vte_cara.pdb [View Structure](http://www.charmm-gui.org/?doc=visualization.ngl&jobid=3vte_cara.pdb&filename=3vte_heta.pdb&project=pdbreader&arg=1)
>>
>> - 3vte_carc.pdb [View Structure](http://www.charmm-gui.org/?doc=visualization.ngl&jobid=3vte_carc.pdb&filename=3vte_heta.pdb&project=pdbreader&arg=1)
>>
>> - 3vte_care.pdb [View Structure](http://www.charmm-gui.org/?doc=visualization.ngl&jobid=3vte_care.pdb&filename=3vte_heta.pdb&project=pdbreader&arg=1)
>>
>> - 3vte_heta.pdb [View Structure](http://www.charmm-gui.org/?doc=visualization.ngl&jobid=3vte_heta.pdb&filename=3vte_heta.pdb&project=pdbreader&arg=1)
>>
>> - 3vte_carb.pdb [View Structure](http://www.charmm-gui.org/?doc=visualization.ngl&jobid=3vte_carb.pdb&filename=3vte_heta.pdb&project=pdbreader&arg=1)
>>
>> - 3vte_card.pdb [View Structure](http://www.charmm-gui.org/?doc=visualization.ngl&jobid=3vte_card.pdb&filename=3vte_heta.pdb&project=pdbreader&arg=1)
>>
>> - 3vte_carf.pdb [View Structure](http://www.charmm-gui.org/?doc=visualization.ngl&jobid=3vte_carf.pdb&filename=3vte_heta.pdb&project=pdbreader&arg=1)
>>

## Component Prepocessing
### 3VTE Apo-protein

### FAD Cofactor
The final FAD structure [HETA.red50.pdb](structures/HETA.red50.pdb) was created from <3vte_heta.pdb>, which lacks hydrogens. All atoms were saturated regardless of H-bonds or pKa. Then, the two covalent FAD bonds at His114 and Cys176 were added with a [tleap command](./FAD-bond.cmd) that replaces a single hydrogen of each residue.
> NOTE: This is a lazy approach that yields misleading results

### NAG Glycans
The 3VTE holoenzyme is glycosylated in-vivo, and a few sites are known to substantially affect catalysis.
However, to lighten computational load, I excluded all sugars from the final complex and replaced the missing glycosidic bonds with hydrogen without considering H-bonds or pH.

### CBGA Ligand
I don't remember where CBGA.pdb came from, but there should be a conformer on PubChem aleady.       
I think I loaded a SMILES string with RDKit and used one of the optimizers; the difference is inconsequential.
> NOTE: CBGA.pdb and CBGA.gaff.mol2 do not represent bound conformations

### Exclusions
Components excluded from the initial complex:
- PDBID: 3VTE
    - All glycans (NAG)
    - All crystallographic waters (HOH)

## Component Parameters
### FAD Cofactor
I'm using the FADH(-) params originally reported by [(Stuchebrukhov, 2000)](https://doi.org/10.1021/ja993784t) and hosted at the Bryce lab [AMBER parameter database](http://research.bmh.manchester.ac.uk/bryce/amber/).
- [FADH-.lib](http://personalpages.manchester.ac.uk/staff/Richard.Bryce/amber/cof/FADH-.lib)
- [FADH-.ff](http://personalpages.manchester.ac.uk/staff/Richard.Bryce/amber/cof/FADH-.frcfld)

The FAD atom names generated by PDB Reader don't match those in the FADH(-) param files, so I use a [tleap command](FAD-bond.cmd) to set each atom type individually.

## Complex Assembly
The tleap script [complex.in] generates coordinate ([complex.inpcrd]) and topology ([complex.prmtop]) files that define the inital 3VTE + FAD + CBGA complex as a single unit in a solvent box.

#### Complex components and associated params:        
Apo-protein
- UNIT = PROA
- RESNAME =
- [PROA.red50.pdb](structurers/PROA.pdb)
    - leaprc.protein.ff14SB

FAD     
- UNIT = FAD        
- RESNAME =         
- [HETA.red50.pdb](structures/HETA.pdb)     
    - FAD-.ff       
    - FAD-.lib      

CBGA        
- UNIT = CBGA       
- RESNAME =         
- [CBGA.gaff.mol2](structures/CBGA.gaff.mol2)       
    - leaprc.gaff2      
        
# Production Model
## Minimization
## Equilibration


In [ ]:
!antechamber \
    -fi pdb -i structures/CBGA.pdb \
    -fo mol2 -o structures/CBGA.gaff.mol2 \
    -rn CBG1 -c bcc -at gaff2 -j 4 -pf y
!parmchk2 \
    -i structures/CBGA.gaff.mol2 \
    -f mol2 -o CBGA.frcmod -s 2

In [ ]:
# !antechamber -fi ccif -i FAD.cif -bk FAD -fo ac -o FAD.ac -c bcc -at amber
# !prepgen -i FAD.ac -o FAD.prepin -m FAD.mc -rn FAD
# !parmchk2 -i FAD.prepin -f prepi -o FAD.frcmod -a Y -p parm10.dat
# !grep -v "ATTN" FAD.frcmod > FAD.1.frcmod
# !parmchk2 -i FAD.prepin -f prepi -o FAD.2.frcmod

In [ ]:
!tleap -f complex.in

In [ ]:
%%bash
# Energy minimization
sander -O \
    -i em.emin.in \
    -p complex.prmtop -c complex.inpcrd \
    -r emin.em.emin.rst

ambpdb -p complex.prmtop -c complex.inpcrd < em.emin.rst > em.emin.pdb

In [ ]:
%%bash
# Graded initial heating
sander -O 
    -i em.heat.in -o heat.log \
    -p gfp.parm7 -c min1.rst7 \
    -r heat.rst -x heat.crd

ambpdb -p complex.prmtop -c complex.inpcrd < emin.em00.rst > emin.em00.pdb

In [ ]:
%%bash
# Restrained equilibration 
sander -O \
    -i emin.eq.in -o emin.eq.log -inf emin.eq.info \
    -p complex.prmtop -c complex.inpcrd \
    -r emin.eq.rst -x emin.eq.crd 